# SATRE feature questionnaire (intermediate summary)

This is an initial intermediate overview of the feature survey results.
It may contain sensitive information, and should not be made public without further review.

This notebook can be executed from the command line and exported as HTML:
```
python -mnbconvert --to html --execute summary-so-far.ipynb
```

Results were exported from JISC online surveys with these options:
- Include date ~~and time~~ of response submission
- Include section headings
- Microsoft Excel 2007 and later (.xlsx)

In [ ]:
from datetime import datetime 
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
# Requires openpyxl for read_excel
from pathlib import Path
import re
from IPython.display import Markdown

In [ ]:
# Import spreadsheet cleaning loader
from load_survey_data import load_data, headers

Choose a spreadsheet to load

In [ ]:
sheet = "summarise-all-results-2023-05-16-1359.xlsx"
format = "%Y-%m-%d %H:%M:%S %Z"

Each column in the spreadsheet should be either:
- Section heading (no data)
- Free text
- Category/ranking question
- Timestamp

In [ ]:
df = load_data(sheet, format)

## Quick overview of completions per day

In [ ]:
completion_dt = df['CompletionDate'].dt.date
start_date = completion_dt.min()
days_since_start = pd.Series((completion_dt - start_date).dt.days)
ax = days_since_start.hist(bins=(days_since_start.max()))
ax.set_xlabel("Days since start of survey (" + start_date.strftime("%a %d %b %Y") + ")")
ax.set_ylabel("Number of responses")
ax.set_title(f"Total: {len(df)} responses");

## Quickly show all the data

- Rankings are shown as bar plots, all use the same X and Y axes
- Free text answers are in alphabetical order

In [ ]:
def truncate_with_ellipsis(s, n):
    if len(s) > n:
        s = s[:(n - 1)]
        return f"{s}…"
    return s


def show_text_answer(column):
    values = sorted(v.strip() for v in column.dropna().values if v.strip())
    display(Markdown(f"### {h} ({len(values)} answers)"))
    display(Markdown("\n".join(f"- {v}" for v in values)))


def show_ranking_answer(column, ax, y_max):
    counts = column.value_counts().sort_index()
    counts.plot(kind='bar', ax=ax)
    ax.set_ylim(0, y_max + 1)
    ax.grid(axis='y')
    # ax.tick_params(axis="x", labelrotation='auto')
    ax.set_ylabel(truncate_with_ellipsis(h, 55))
    return ax


# Maximum number for any answer
y_max = 0
for (h, t) in headers.items():
    if t == "RANKING":
        y_max = max(y_max, df[h].value_counts().max())

max_plots_in_one_section = 5

matplotlib.rcParams.update({'figure.max_open_warning': 0})

fig = None
axs = []
ax_n = 0
for (h, t) in headers.items():
    if t != "RANKING":
        # Delete unused plot spaces
        for ax in axs[ax_n:]:
            fig.delaxes(ax)
        # Show plot NOW, so that we can interleave plots and markdown
        if ax_n:
            plt.show()
        fig = None
        axs = []
        ax_n = 0

    if t == "SECTION":
        display(Markdown(f"## {h}"))
    if t == "TEXT":
        show_text_answer(df[h])
    if t == "RANKING":
        if not fig:
            fig, axs = plt.subplots(1, max_plots_in_one_section, figsize=(10, 5))
            fig.subplots_adjust(wspace=1)
        ax = axs[ax_n]
        show_ranking_answer(df[h], ax, y_max)
        ax_n += 1
